In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Aug  3 14:30:42 2023

@author: taylo
"""

#%%Importing modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy.table import Table
from astropy.convolution import convolve, Gaussian1DKernel

#%%DESI targeting masks and DataLab related modules
from desitarget.sv1 import sv1_targetmask    # For SV1
from desitarget.sv2 import sv2_targetmask    # For SV2
from desitarget.sv3 import sv3_targetmask    # For SV3

from sparcl.client import SparclClient
from dl import queryClient as qc
from spender.data import desi

#%%Load SPARCLClient and DESI Module
client = SparclClient()
desi = desi.DESI()

#%%DESI EDR Details
query = """
SELECT zp.targetid, zp.survey, zp.program, zp.healpix,  
       zp.z, zp.zwarn, zp.coadd_fiberstatus, zp.spectype, 
       zp.mean_fiber_ra, zp.mean_fiber_dec, zp.zcat_nspec, 
       CAST(zp.zcat_primary AS INT), zp.desi_target,
       zp.sv1_desi_target, zp.sv2_desi_target, zp.sv3_desi_target
FROM desi_edr.zpix AS zp
"""
zpix = qc.query(sql = query, fmt='pandas')
#%%
target_id = zpix['targetid'].unique()

sv1_desi_tgt = zpix['sv1_desi_target']
sv2_desi_tgt = zpix['sv2_desi_target']
sv3_desi_tgt = zpix['sv3_desi_target']

is_sv1 = (sv1_desi_tgt != 0)
is_sv2 = (sv2_desi_tgt != 0)
is_sv3 = (sv3_desi_tgt != 0)


healpix = zpix['healpix'].unique()
healpix = list(healpix)



In [2]:
#%%
import requests
import os
hard_drive_path = os.path.abspath(os.sep)
print("Path to your hard drive:", hard_drive_path)
#%%

def download_files(base_url, file_types, surveys, programmes, save_directory):
    for survey in surveys:
        z_survey = zpix.copy()
        survey_dir = os.path.join(save_directory, survey)
        if survey == 'sv3':
            z_survey = z_survey[is_sv3]
        elif survey == 'sv2':
            z_survey = z_survey[is_sv2]
        elif survey == 'sv1':
            z_survey = z_survey[is_sv1]
        elif survey == 'special':
            z_survey == z_survey[z_survey['survey'] == 'special']
        else:
            z_survey == z_survey[z_survey['survey'] == 'cmx']
        
        for programme in programmes:
            programme_dir = os.path.join(survey_dir, programme)
            z_program = z_survey.copy()
            
            if programme == 'dark':
                z_program = z_program[z_program['program'] == 'dark']
            elif programme == 'bright':
                z_program = z_program[z_program['program'] == 'bright']
            elif programme == 'backup':
                z_program = z_program[z_program['program'] == 'backup']
            else:
                z_program = z_program[z_program['program'] == 'other']
            
            healpix_values = z_program['healpix'].unique()
            
            for healpix in healpix_values:
                healpix_str = str(healpix)  #Convert to string to get the whole healpix number
                if len(healpix_str) == 3:
                    healpix_dir = os.path.join(programme_dir, healpix_str[:1])
                    short_h = healpix_str[:1]
                elif len(healpix_str) == 4:
                    healpix_dir = os.path.join(programme_dir, healpix_str[:2])
                    short_h = healpix_str[:2]
                elif len(healpix_str) == 5:
                    healpix_dir = os.path.join(programme_dir, healpix_str[:3])
                    short_h = healpix_str[:3]
                else:
                    raise ValueError(f"Invalid healpix number: {healpix_str}")

                full_healpix_dir = os.path.join(healpix_dir, healpix_str)
                os.makedirs(full_healpix_dir, exist_ok=True)

                url = os.path.join(base_url, survey, programme, short_h, healpix_str)

                for file_type in file_types:
                    file_url = os.path.join(url, f"{file_type}-{survey}-{programme}-{healpix_str}.h5" if file_type == "rrdetails" else f"{file_type}-{survey}-{programme}-{healpix_str}.fits")
                    save_path = os.path.join(full_healpix_dir, f"{file_type}-{survey}-{programme}-{healpix_str}.h5" if file_type == "rrdetails" else f"{file_type}-{survey}-{programme}-{healpix_str}.fits")
                    download_file(file_url, save_path)

def download_file(url, save_path):
    
    if os.path.exists(save_path):
        print(f"File already exists: {save_path}")
        return
    
    try:
        response = requests.get(url)
        response.raise_for_status()  
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {save_path}")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {url}: {e}")

#%%
# Example usage:
base_url = "https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/healpix/"
file_types = ["coadd"] #can have ["redrock", "rrdetails", "emline", "qso_mgii", "qso_qn", "coadd"]
surveys = ["sv1", "sv2", "sv3"] #can have["sv1", "sv2", "sv3", "special", "cmx"]
programmes = ["dark", "bright", "other", "backup"]
mount_point = "/Volumes/DESI-Data"  # The mount point of your hard drive
save_directory = os.path.join(mount_point, "DESI-Data")



Path to your hard drive: /


In [ ]:
# Create the base directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

download_files(base_url, file_types, surveys, programmes, save_directory)


File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/114/11417/coadd-sv1-dark-11417.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/114/11418/coadd-sv1-dark-11418.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/114/11416/coadd-sv1-dark-11416.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/114/11419/coadd-sv1-dark-11419.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/114/11440/coadd-sv1-dark-11440.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/114/11442/coadd-sv1-dark-11442.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/114/11431/coadd-sv1-dark-11431.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/114/11405/coadd-sv1-dark-11405.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/114/11407/coadd-sv1-dark-11407.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/114/11429/coadd-sv1-dark-11429.fits
File already exists: /Volumes/DESI-Data/DESI-Data/

File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/75/7587/coadd-sv1-dark-7587.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/75/7585/coadd-sv1-dark-7585.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/75/7566/coadd-sv1-dark-7566.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/75/7586/coadd-sv1-dark-7586.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/75/7563/coadd-sv1-dark-7563.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/75/7584/coadd-sv1-dark-7584.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/75/7564/coadd-sv1-dark-7564.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/75/7559/coadd-sv1-dark-7559.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/75/7558/coadd-sv1-dark-7558.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/75/7562/coadd-sv1-dark-7562.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/75/7561/coadd-sv1-dar

File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/176/17687/coadd-sv1-dark-17687.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/176/17685/coadd-sv1-dark-17685.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/177/17728/coadd-sv1-dark-17728.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/177/17736/coadd-sv1-dark-17736.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/177/17730/coadd-sv1-dark-17730.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/177/17738/coadd-sv1-dark-17738.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/177/17737/coadd-sv1-dark-17737.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/177/17731/coadd-sv1-dark-17731.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/177/17739/coadd-sv1-dark-17739.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/177/17761/coadd-sv1-dark-17761.fits
File already exists: /Volumes/DESI-Data/DESI-Data/

File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/68/6807/coadd-sv1-dark-6807.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/68/6850/coadd-sv1-dark-6850.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/68/6856/coadd-sv1-dark-6856.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/68/6849/coadd-sv1-dark-6849.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/66/6685/coadd-sv1-dark-6685.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/67/6722/coadd-sv1-dark-6722.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/67/6720/coadd-sv1-dark-6720.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/67/6728/coadd-sv1-dark-6728.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/67/6730/coadd-sv1-dark-6730.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/67/6737/coadd-sv1-dark-6737.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/67/6738/coadd-sv1-dar

File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/50/5052/coadd-sv1-dark-5052.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/50/5054/coadd-sv1-dark-5054.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/50/5046/coadd-sv1-dark-5046.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/50/5045/coadd-sv1-dark-5045.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/50/5047/coadd-sv1-dark-5047.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/50/5053/coadd-sv1-dark-5053.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/50/5020/coadd-sv1-dark-5020.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/50/5023/coadd-sv1-dark-5023.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/50/5021/coadd-sv1-dark-5021.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/50/5088/coadd-sv1-dark-5088.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv1/dark/50/5066/coadd-sv1-dar

...

File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/153/15341/coadd-sv3-bright-15341.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/153/15352/coadd-sv3-bright-15352.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/153/15342/coadd-sv3-bright-15342.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/153/15339/coadd-sv3-bright-15339.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/153/15340/coadd-sv3-bright-15340.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/153/15343/coadd-sv3-bright-15343.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/153/15338/coadd-sv3-bright-15338.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/153/15353/coadd-sv3-bright-15353.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/153/15354/coadd-sv3-bright-15354.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/153/15355/coadd-sv3-bright-15355.fits
File alrea

File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/116/11612/coadd-sv3-bright-11612.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/119/11912/coadd-sv3-bright-11912.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/119/11914/coadd-sv3-bright-11914.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/112/11231/coadd-sv3-bright-11231.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/119/11936/coadd-sv3-bright-11936.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/112/11253/coadd-sv3-bright-11253.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/112/11252/coadd-sv3-bright-11252.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/98/9829/coadd-sv3-bright-9829.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/98/9831/coadd-sv3-bright-9831.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/98/9830/coadd-sv3-bright-9830.fits
File already exists

File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/262/26285/coadd-sv3-bright-26285.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/262/26279/coadd-sv3-bright-26279.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/262/26287/coadd-sv3-bright-26287.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/262/26290/coadd-sv3-bright-26290.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/262/26296/coadd-sv3-bright-26296.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/281/28120/coadd-sv3-bright-28120.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/281/28114/coadd-sv3-bright-28114.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/281/28112/coadd-sv3-bright-28112.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/281/28115/coadd-sv3-bright-28115.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/bright/281/28121/coadd-sv3-bright-28121.fits
File alrea

File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/153/15356/coadd-sv3-backup-15356.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/153/15358/coadd-sv3-backup-15358.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/153/15355/coadd-sv3-backup-15355.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/102/10231/coadd-sv3-backup-10231.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/153/15357/coadd-sv3-backup-15357.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/160/16040/coadd-sv3-backup-16040.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/153/15359/coadd-sv3-backup-15359.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/160/16041/coadd-sv3-backup-16041.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/160/16042/coadd-sv3-backup-16042.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/160/16043/coadd-sv3-backup-16043.fits
File alrea

File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/99/9922/coadd-sv3-backup-9922.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/98/9877/coadd-sv3-backup-9877.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/98/9879/coadd-sv3-backup-9879.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/98/9885/coadd-sv3-backup-9885.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/99/9926/coadd-sv3-backup-9926.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/99/9929/coadd-sv3-backup-9929.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/99/9931/coadd-sv3-backup-9931.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/99/9928/coadd-sv3-backup-9928.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/99/9930/coadd-sv3-backup-9930.fits
File already exists: /Volumes/DESI-Data/DESI-Data/sv3/backup/98/9887/coadd-sv3-backup-9887.fits
File already exists: /Volumes/DESI-Data/